In [61]:
import pandas as pd
import sys
import matplotlib.dates as mdates
import plotly.graph_objects as go

In [62]:
sys.path.append('/home/jeffreydhy/workspace/project-ares/ares-finance')

In [63]:
from data.fetcher.polygon_data_fetcher import PolygonDataFetcher
from utils.time_utils import Utility
from utils.data_utils import DataUtils

In [64]:
data_path = '/home/jeffreydhy/data/projects/ares-finance/raw/'
date = '2023-03-23'
raw_trade_data = pd.read_csv('{}amd_{}.csv'.format(data_path, date))
one_min_data = pd.read_csv('{}amd_{}_1_min.csv'.format(data_path, date), parse_dates=['timestamp'])


In [65]:
one_min_data.head()

,timestamp,open,close,high,low,v,vw
0,2023-03-23 09:30:00-04:00,100.1100,99.730,100.4000,99.69,1295846,100.085600
1,2023-03-23 09:31:00-04:00,99.7582,99.685,99.9750,99.55,277389,99.693073
2,2023-03-23 09:32:00-04:00,99.6600,100.400,100.4400,99.56,465222,100.093133
3,2023-03-23 09:33:00-04:00,100.4100,100.000,100.5400,99.80,510676,100.232898
4,2023-03-23 09:34:00-04:00,100.0000,99.685,100.0421,99.59,333304,99.706820


In [70]:
fig = go.Figure(data=[go.Candlestick(x=one_min_data['timestamp'],
                open=one_min_data['open'],
                high=one_min_data['high'],
                low=one_min_data['low'],
                close=one_min_data['close'], increasing_line_color='green', decreasing_line_color='red')])

# Remove slider, set background color to white and remove grid
fig.update_layout(xaxis_rangeslider_visible=False, paper_bgcolor='white')


In [74]:
def find_consolidation_and_breakout(df, min_consolidation_bars=3, max_consolidation_bars=10, volume_multiplier=1.5, post_breakout_bars=1):
    consolidation_ranges = []
    breakouts = []

    for bar_count in range(min_consolidation_bars, max_consolidation_bars + 1):
        for i in range(len(one_min_data) - bar_count - post_breakout_bars):
            current_bars = df[i:i + bar_count]
            next_bar = df.iloc[i + bar_count]
            post_breakout_bars_data = df[i + bar_count + 1:i + bar_count + 1 + post_breakout_bars]

            min_price = current_bars['low'].min()
            max_price = current_bars['high'].max()
            price_range = max_price - min_price
            percentage_range = price_range / min_price * 100

            if (next_bar['high'] > max_price) and (next_bar['v'] >= volume_multiplier * current_bars['v'].mean()):
                if post_breakout_bars_data['close'].min() > next_bar['close']:
                    consolidation_end = current_bars.iloc[-1]['timestamp']
                    breakout_time = next_bar['timestamp']
                    breakouts.append((consolidation_end, breakout_time, bar_count, percentage_range))
                    consolidation_ranges.append((current_bars.iloc[0]['timestamp'], consolidation_end, bar_count, percentage_range))
                
    return consolidation_ranges, breakouts

In [75]:
one_min_data['timestamp'] = pd.to_datetime(one_min_data['timestamp'])

consolidation_ranges, breakouts = find_consolidation_and_breakout(one_min_data)
print("Consolidation Ranges:", consolidation_ranges)
print("Breakouts:", breakouts)

Consolidation Ranges: [(Timestamp('2023-03-23 09:54:00-0400', tz='UTC-04:00'), Timestamp('2023-03-23 09:56:00-0400', tz='UTC-04:00'), 3, 0.289132602193426), (Timestamp('2023-03-23 09:55:00-0400', tz='UTC-04:00'), Timestamp('2023-03-23 09:57:00-0400', tz='UTC-04:00'), 3, 0.47851659854451595), (Timestamp('2023-03-23 10:21:00-0400', tz='UTC-04:00'), Timestamp('2023-03-23 10:23:00-0400', tz='UTC-04:00'), 3, 0.3165182987141517), (Timestamp('2023-03-23 10:36:00-0400', tz='UTC-04:00'), Timestamp('2023-03-23 10:38:00-0400', tz='UTC-04:00'), 3, 0.26306413301662573), (Timestamp('2023-03-23 10:57:00-0400', tz='UTC-04:00'), Timestamp('2023-03-23 10:59:00-0400', tz='UTC-04:00'), 3, 0.36460386283010476), (Timestamp('2023-03-23 11:44:00-0400', tz='UTC-04:00'), Timestamp('2023-03-23 11:46:00-0400', tz='UTC-04:00'), 3, 0.177514792899401), (Timestamp('2023-03-23 12:05:00-0400', tz='UTC-04:00'), Timestamp('2023-03-23 12:07:00-0400', tz='UTC-04:00'), 3, 0.235640648011791), (Timestamp('2023-03-23 13:17:00-